# ResNet50 Global Wheat Detection 

1. Plotting same sample
2. Splitting Train and Validation set
3. Creating Image generator
4. Data Augmentation
5. Model Creating
6. Model Training
7. Model Evaluation 




In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from tensorflow.keras.applications.resnet  import  ResNet50 as resNet
from tensorflow.keras.models import Sequential
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as albu

### 1. Plotting same sample

In [ ]:
img_width,img_height = 512,512

root_path = "/kaggle/input/global-wheat-detection"
train_path = root_path+"/train"
test_path =  root_path+"/test"
train_csv_path =  root_path+"/train.csv"
sample_path =  root_path+"/sample_submission.csv"

In [ ]:
total_df = pd.read_csv(train_csv_path)
total_ids =  [i.split(".")[0] for i in  os.listdir(train_path)]
total_df.head()

In [ ]:
def draw_bboxes(bboxs,img):
    color = (255, 0, 0) 
    thickness = 3
    for cur_box  in bboxs:
        start_point = (cur_box[0],cur_box[1])
        end_point = (cur_box[2]+cur_box[0],cur_box[3]+cur_box[1]) 
        cv2.rectangle(img, start_point, end_point, color, thickness) 

    return img

In [ ]:
def get_bboxes(image_id):
    selected_df = total_df[total_df['image_id'] == image_id]
    
    image_bboxes = selected_df['bbox']
    box_scores = selected_df['bbox']
    bboxes = []
    for row in image_bboxes:
        row=row.replace(" ", "")
        x1y1x2y2 = np.array(re.findall("([0-9]+[.]?[0-9]*)", row))
        x1y1x2y2 = np.float32(x1y1x2y2)
        x1y1x2y2 = x1y1x2y2/2
        x1y1x2y2 =  np.int32(x1y1x2y2)
  
        bboxes.append(x1y1x2y2)
           
        
    return np.array(bboxes)

In [ ]:
def read_image(image_id):
    path = train_path+'/'+str(image_id)+'.jpg'
    image = Image.open(path)
    image = image.resize((img_width,img_height))
    
    return np.asarray(image)
    

In [ ]:
N = 10
image_id = total_ids[N]
selected_df = total_df.loc[total_df['image_id'] == image_id]
bboxes = get_bboxes(image_id)
sample_img  = read_image(image_id)
box_img =  draw_bboxes(bboxes,sample_img)
plt.imshow(box_img)

In [ ]:
 
rows=3
cols=3
N = 0
fig, axs = plt.subplots(rows, cols, figsize=(30,35))
for row in range(rows):
    for col in range(cols):
        image_id = total_ids[N]
        selected_df = total_df.loc[total_df['image_id'] == image_id]
        read_path = train_path+"\\"+image_id+".jpg"
        sample_img = read_image(image_id)
        


        if selected_df.empty:
            axs[row, col].imshow(sample_img)
            axs[row, col].axis('off')
            
            N = N+1
            continue
    
        
        bboxes = get_bboxes(image_id)
        box_img =  draw_bboxes(bboxes,sample_img)
        axs[row, col].imshow(box_img)
        axs[row, col].axis('off')
        N = N+1

plt.suptitle("Same Examples Images")

### 2. Splitting Train and Validation set

In [ ]:
total_amount = len(os.listdir(train_path))
split_rate = 0.98
train_amount = int(total_amount * split_rate)
test_amount = total_amount - train_amount
print("Train amount : ",train_amount)
print("Test amount : ",test_amount)


In [ ]:
train_ids = total_ids[:train_amount]
val_ids = total_ids[train_amount:]

In [ ]:
train_images = {}
train_labels = {}
for image_id in train_ids:
    bboxes = get_bboxes(image_id)
    train_labels[image_id] = bboxes
    train_images[image_id] = read_image(image_id)
    


In [ ]:
val_images = {}
val_labels = {}
for image_id in val_ids:
    bboxes = get_bboxes(image_id)
    val_labels[image_id] = bboxes
    val_images[image_id] = read_image(image_id)

### 3. Creating Image generator

I am newbie in deep learning and I entered this competition as an opportunity to practice object detection model.
In creating object detection model, I didn't know how to implement loss functions and image generators for model input.
So searched and I found amazing this notebook -> [notebook](https://www.kaggle.com/mattbast/object-detection-tensorflow-end-to-end). 

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, image_ids, image_pixels, labels=None, batch_size=1, shuffle=False, augment=False):
        self.image_ids = image_ids
        self.image_pixels = image_pixels
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()
        
        self.image_grid = self.from_image_grid()
        
    def from_image_grid(self):
        image_grid = np.zeros((32,32,4))
        
        # x, y, width, height
        cell = [0, 0, 256/32, 256/32]
        
        for i in range(0, 32):
            for j in range(0, 32):
                image_grid[i,j] = cell
                
                cell[0] = cell[0] + cell[2]
                
                
            cell[0] = 0
            cell[1] = cell[1] + cell[3]
            
        return image_grid
        

### 4. Data Augmentation

In [ ]:
DataGenerator.train_augmentations = albu.Compose([
    albu.RandomSizedCrop(
            min_max_height=(200, 200), 
            height=img_height, 
            width=img_width, 
            p=0.8
        ),
    albu.OneOf([
            albu.Flip(),
            albu.RandomRotate90(),
        ], p=1),
    albu.OneOf([
            albu.HueSaturationValue(),
            albu.RandomBrightnessContrast()
        ], p=1),
    albu.OneOf([
            albu.GaussNoise(),
            albu.GlassBlur(),
            albu.ISONoise(),
            albu.MultiplicativeNoise(),
        ], p=0.5),
    albu.Cutout(
            num_holes=8, 
            max_h_size=16, 
            max_w_size=16, 
            fill_value=0, 
            p=0.5
        ),
        albu.CLAHE(p=1),
        albu.ToGray(p=1),
    ], 
    bbox_params={'format': 'coco', 'label_fields': ['labels']})

DataGenerator.val_augmentations = albu.Compose([
    albu.CLAHE(p=1),
    albu.ToGray(p=1),
])

In [ ]:
def __len__(self):
    return int(np.floor(len(self.image_ids) / self.batch_size))

def on_epoch_end(self):
    self.indexes = np.arange(len(self.image_ids))
    
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

DataGenerator.__len__ = __len__ 
DataGenerator.on_epoch_end = on_epoch_end

In [ ]:
def __getitem__(self, index):
    indexes =  self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

    batch_ids = [self.image_ids[i] for i in indexes]

    X, y = self.__data_generation(batch_ids)

  
    return X,y

In [ ]:
def __data_generation(self, batch_ids):
    X, y = [], []

    # Generate data
    for i, image_id in enumerate(batch_ids):
        pixels = self.image_pixels[image_id]
        bboxes = self.labels[image_id]

        if self.augment:     
            pixels, bboxes = self.augment_image(pixels, bboxes)
        else:
            pixels = self.contrast_image(pixels)
            bboxes = self.form_label_grid(bboxes)

        X.append(pixels)
        y.append(bboxes)

    return np.array(X), np.array(y)

In [ ]:
def augment_image(self, pixels, bboxes):
    bbox_labels = np.ones(len(bboxes))

    aug_result = self.train_augmentations(image=pixels, bboxes=bboxes, labels=bbox_labels)
    bboxes = self.form_label_grid(aug_result['bboxes'])
    

    
    return np.array(aug_result['image']) / 255, bboxes

def contrast_image(self, pixels):        
    aug_result = self.val_augmentations(image=pixels)
    return np.array(aug_result['image']) / 255


DataGenerator.__getitem__ = __getitem__
DataGenerator.__data_generation = __data_generation
DataGenerator.augment_image = augment_image
DataGenerator.contrast_image = contrast_image

In [ ]:
def form_label_grid(self, bboxes):
    label_grid = np.zeros((32, 32, 10))

    for i in range(0, 32):
        for j in range(0, 32):
            cell = self.image_grid[i,j]
            label_grid[i,j] = self.rect_intersect(cell, bboxes)

    return label_grid

In [ ]:
def rect_intersect(self, cell, bboxes): 
    cell_x, cell_y, cell_width, cell_height = cell
    cell_x_max = cell_x + cell_width 
    cell_y_max = cell_y + cell_height
    
    anchor_one = np.array([0, 0, 0, 0, 0])
    anchor_two = np.array([0, 0, 0, 0, 0])

    # check all boxes
    for bbox in bboxes:
        box_x, box_y, box_width, box_height = bbox
        box_x_centre = box_x + (box_width / 2)
        box_y_centre = box_y + (box_height / 2)

        if(box_x_centre >= cell_x and box_x_centre < cell_x_max and box_y_centre >= cell_y and box_y_centre < cell_y_max):
            
            if anchor_one[0] == 0:
                anchor_one = self.yolo_shape(
                    [box_x, box_y, box_width, box_height], 
                    [cell_x, cell_y, cell_width, cell_height]
                )
            
            elif anchor_two[0] == 0:
                anchor_two = self.yolo_shape(
                    [box_x, box_y, box_width, box_height], 
                    [cell_x, cell_y, cell_width, cell_height]
                )
                
            else:
                break


    return np.concatenate((anchor_one, anchor_two), axis=None)


In [ ]:
def yolo_shape(self, box, cell):
    box_x, box_y, box_width, box_height = box
    cell_x, cell_y, cell_width, cell_height = cell

    # top left x,y to centre x,y
    box_x = box_x + (box_width / 2)
    box_y = box_y + (box_height / 2)

    # offset bbox x,y to cell x,y
    box_x = (box_x - cell_x) / cell_width
    box_y = (box_y - cell_y) / cell_height

    # bbox width,height relative to cell width,height
    box_width = box_width / img_width
    box_height = box_height / img_height
    return [1, box_x, box_y, box_width, box_height]


DataGenerator.form_label_grid = form_label_grid
DataGenerator.rect_intersect = rect_intersect
DataGenerator.yolo_shape = yolo_shape


In [ ]:
train_generator = DataGenerator(
    train_ids,
    train_images,
    train_labels, 
    batch_size=6, 
    shuffle=True,
    augment=True
)


val_generator = DataGenerator(
    val_ids, 
    val_images,
    val_labels, 
    batch_size=6,
    shuffle=False,
    augment=False
)

image_grid = train_generator.image_grid
image_grid.shape


In [ ]:
print(np.array(train_labels[train_ids[0]]).shape)
print(np.array(train_images[train_ids[0]]).shape)
print(train_ids[0])


In [ ]:
print(val_labels[val_ids[0]].shape)
print(val_images[val_ids[0]].shape)
#print(val_ids)
x,y = train_generator.__getitem__(0)
print(x.shape)
print(y.shape)

In [ ]:
x,y = train_generator.__getitem__(0)

print(x.shape)
print(y.shape)


### 5. Model Creating
#### Why ResNet
When deeper networks starts converging, a degradation problem has been exposed: with the network depth increasing, accuracy gets saturated and then degrades rapidly.

#### So Was ResNet Successful?
* Won 1st place in the ILSVRC 2015 classification competition with top-5 error rate of 3.57% (An ensemble model)
* Won the 1st place in ILSVRC and COCO 2015 competition in ImageNet Detection, ImageNet localization, Coco detection and Coco segmentation.
* Replacing VGG-16 layers in Faster R-CNN with ResNet-101. They observed a relative improvements of 28%
* Efficiently trained networks with 100 layers and 1000 layers also.

If interest for more detail, you should check out this awesome article: [Understanding and Coding a ResNet in Keras](https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33#:~:text=The%20ResNet%2D50%20model%20consists,over%2023%20million%20trainable%20parameters.&text=Our%20ResNet%2D50%20gets%20to,in%2025%20epochs%20of%20training.)
![ResNet Architecture](https://www.codeproject.com/KB/AI/1248963/resnet.png)

In [ ]:
img_input_shape = (512,512,3)
base_model =resNet(include_top=False, weights='imagenet', input_shape=img_input_shape)
base_model.summary()


In [ ]:
base_model_output = base_model.layers[-33].output
x_input = base_model.input
x = tf.keras.layers.Dropout(0.25)(base_model_output)


predictions = tf.keras.layers.Conv2D(10, (1, 1), strides=(1, 1), activation='sigmoid')(x)
model=tf.keras.Model(inputs=x_input,outputs=predictions)
model.summary()

In [ ]:
def custom_loss(y_true, y_pred):
    binary_crossentropy = prob_loss = tf.keras.losses.BinaryCrossentropy(
        reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
    )
    
    prob_loss = binary_crossentropy(
        tf.concat([y_true[:,:,:,0], y_true[:,:,:,5]], axis=0), 
        tf.concat([y_pred[:,:,:,0], y_pred[:,:,:,5]], axis=0)
    )
    
    xy_loss = tf.keras.losses.MSE(
        tf.concat([y_true[:,:,:,1:3], y_true[:,:,:,6:8]], axis=0), 
        tf.concat([y_pred[:,:,:,1:3], y_pred[:,:,:,6:8]], axis=0)
    )
    
    wh_loss = tf.keras.losses.MSE(
        tf.concat([y_true[:,:,:,3:5], y_true[:,:,:,8:10]], axis=0), 
        tf.concat([y_pred[:,:,:,3:5], y_pred[:,:,:,8:10]], axis=0)
    )
    
    bboxes_mask = get_mask(y_true)
    
    xy_loss = xy_loss * bboxes_mask
    wh_loss = wh_loss * bboxes_mask
    
    return prob_loss + xy_loss + wh_loss

In [ ]:
def get_mask(y_true):
    anchor_one_mask = tf.where(
        y_true[:,:,:,0] == 0, 
        0.5, 
        5.0
    )
    
    anchor_two_mask = tf.where(
        y_true[:,:,:,5] == 0, 
        0.5, 
        5.0
    )
    
    bboxes_mask = tf.concat(
        [anchor_one_mask,anchor_two_mask],
        axis=0
    )
    
    return bboxes_mask

In [ ]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimiser, 
    loss=custom_loss,
    metrics = ['accuracy']
)

### 6. Model Training

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    epochs=2,
    
)

In [ ]:
model.save("test_model.h5")

In [ ]:
model = tf.keras.models.load_model("test_model.h5")

### 7. Model Evaluation 

In [ ]:
def prediction_to_bbox(bboxes, image_grid):    
    bboxes = bboxes.copy()
    
    im_width = (image_grid[:,:,2] * 32)
    im_height = (image_grid[:,:,3] * 32)
    
    # descale x,y
    bboxes[:,:,1] = (bboxes[:,:,1] * image_grid[:,:,2]) + image_grid[:,:,0]
    bboxes[:,:,2] = (bboxes[:,:,2] * image_grid[:,:,3]) + image_grid[:,:,1]
    bboxes[:,:,6] = (bboxes[:,:,6] * image_grid[:,:,2]) + image_grid[:,:,0]
    bboxes[:,:,7] = (bboxes[:,:,7] * image_grid[:,:,3]) + image_grid[:,:,1]
    
    # descale width,height
    bboxes[:,:,3] = bboxes[:,:,3] * im_width 
    bboxes[:,:,4] = bboxes[:,:,4] * im_height
    bboxes[:,:,8] = bboxes[:,:,8] * im_width 
    bboxes[:,:,9] = bboxes[:,:,9] * im_height
    
    # centre x,y to top left x,y
    bboxes[:,:,1] = bboxes[:,:,1] - (bboxes[:,:,3] / 2)
    bboxes[:,:,2] = bboxes[:,:,2] - (bboxes[:,:,4] / 2)
    bboxes[:,:,6] = bboxes[:,:,6] - (bboxes[:,:,8] / 2)
    bboxes[:,:,7] = bboxes[:,:,7] - (bboxes[:,:,9] / 2)
    
    # width,heigth to x_max,y_max
    bboxes[:,:,3] = bboxes[:,:,1] + bboxes[:,:,3]
    bboxes[:,:,4] = bboxes[:,:,2] + bboxes[:,:,4]
    bboxes[:,:,8] = bboxes[:,:,6] + bboxes[:,:,8]
    bboxes[:,:,9] = bboxes[:,:,7] + bboxes[:,:,9]
    
    return bboxes



In [ ]:
def process_predictions(predictions, image_ids, image_grid):
    bboxes = {}
    
    for i, image_id in enumerate(image_ids):
        #print(predictions[i])

        predictions[i] = prediction_to_bbox(predictions[i], image_grid)
        bboxes[image_id] = non_max_suppression(predictions[i], top_n=100)
        
        # back to coco shape
        bboxes[image_id][:,2:4] = bboxes[image_id][:,2:4] - bboxes[image_id][:,0:2]
    
    return bboxes

In [ ]:
def non_max_suppression(predictions, top_n):
    probabilities = np.concatenate((predictions[:,:,0].flatten(), predictions[:,:,5].flatten()), axis=None)
    
    first_anchors = predictions[:,:,1:5].reshape((32*32, 4))
    second_anchors = predictions[:,:,6:10].reshape((32*32, 4))
    
    bboxes = np.concatenate(
        (first_anchors,second_anchors),
        axis=0
    )
    bboxes = switch_x_y(bboxes)
   
    bboxes, probabilities = select_top(probabilities, bboxes, top_n=top_n)
    bboxes = switch_x_y(bboxes)
    
    return bboxes


def switch_x_y(bboxes):
    x1 = bboxes[:,0].copy()
    y1 = bboxes[:,1].copy()
    x2 = bboxes[:,2].copy()
    y2 = bboxes[:,3].copy()
    
    bboxes[:,0] = y1
    bboxes[:,1] = x1
    bboxes[:,2] = y2
    bboxes[:,3] = x2
    
    return bboxes


def select_top(probabilities, boxes, top_n=10):
    top_indices = tf.image.non_max_suppression(
        boxes = boxes, 
        scores = probabilities, 
        max_output_size = top_n, 
        iou_threshold = 0.3,
        score_threshold = 0.3
    )
    
    top_indices = top_indices.numpy()
    print(top_indices.shape)
    return boxes[top_indices], probabilities[top_indices]

In [ ]:
model_predictions = model.predict(val_generator)
print("Model output : ",model_predictions.shape)



In [ ]:
#val_ids = val_ids[:-1]

In [ ]:

val_predictions = process_predictions(model_predictions, val_ids, image_grid)
model_predictions[0].shape


In [ ]:
 
rows=5
cols=5
N = 0
fig, axs = plt.subplots(rows, cols, figsize=(30,35))
for row in range(rows):
    for col in range(cols):
        image_id = val_ids[N]
        sample_img = read_image(image_id)
  


        if selected_df.empty:
            axs[row, col].imshow(sample_img)
            axs[row, col].axis('off')
            
            N = N+1
            continue
    
        
        bboxes = val_predictions[image_id]
        #bboxes = np.square(bboxes)/255
        print(bboxes)
        box_img =  draw_bboxes(bboxes*4,sample_img)
        axs[row, col].imshow(box_img)
        axs[row, col].axis('off')
        N = N+1

plt.suptitle("Same Val Examples Images")

In [ ]:
 import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
val_predictions['b67b4d2e8'].reshape((4,0))[0]

In [ ]:
x,y,width,height = val_predictions['b67b4d2e8']

In [ ]:
sample_df = pd.read_csv(sample_path)
sample_df.head()